In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


In [7]:
path = "/content/drive/MyDrive/glove.6B.300d.txt"
wv = pd.read_table(path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [9]:
def vec(w):
  return wv.loc[w].as_matrix()

In [10]:
def w2v_tokenize_text(text):
  tokens = []
  for sent in nltk.sent_tokenize(text, language='english'):
    for word in nltk.word_tokenize(sent, language='english'):
      if len(word) < 2:
        continue
      tokens.append(word)
  return tokens

In [11]:
def wordprint(words,wv):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.index:
           a=np.array(wv.loc[word])
           mean.append(a)
    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(300,)   
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [12]:
def  word_averaging_listn(wv, text_list):
  return np.vstack([wordprint(post,wv) for post in text_list ])

In [23]:
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [24]:
df.iloc[:,1] = df.iloc[:,1].apply(clean_text)
print(df.iloc[:100,1])

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [25]:
comtdata=df

In [31]:
test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r.iloc[1]), axis=1).values

In [32]:
X_comtdata_averagen=word_averaging_listn(wv, test_tokenized)

In [33]:
len(X_comtdata_averagen)

1744

In [34]:
type(X_comtdata_averagen)

numpy.ndarray

In [35]:
X_comtdata_averagen = pd.DataFrame(X_comtdata_averagen)

In [36]:
from google.colab import files
X_comtdata_averagen.to_csv('Glove_algebra.csv') 
files.download('Glove_algebra.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>